## Rules

In [1]:
sentence_rules='KB/rush_rules.tsv'

In [2]:
with open('KB/pe_targets.yml','r') as f:     
    target_rules = f.read()
with open('KB/pe_modifiers.yml','r') as f: 
    context_rules = f.read()
with open('KB/pe_featurer_inferences.csv','r') as f:     
    feature_inference_rule = f.read()
with open('KB/pe_doc_inferences.csv','r') as f:  
    document_inference_rule = f.read()

## NLP pipeline

In [3]:
from pynlp_pipe_pe import Mypipe

In [4]:
myPipe=Mypipe(sentence_rules, target_rules, context_rules, feature_inference_rule, document_inference_rule)

## Read filenames

In [5]:
# pe notes
import os
unid = 'u0496358'
path = "/home/"+str(unid)+"/BRAT/"+str(unid)+"/Project_pe_test" 
files = os.listdir(path)
len(files)


44

## Read documents and apply pipeline

In [6]:
import chardet

results=dict()  
doc_annotations=dict()
context_doc_res=[]

note_count = 0                       # count the number of text notes want to process ***
for i in files[:]:
    if ".txt" in i:                  
        note_count = note_count + 1  
        #if note_count > 2:          # count the number of text notes want to process ***
        #    break                   
        with open(os.path.join(path,i), 'rb') as f:
            doc_txt = chardet.detect(f.read())        
        
        with open(os.path.join(path,i),encoding=doc_txt["encoding"]) as f:
            doc_text = f.read()           
            #doc_text=doc_text.replace('\n', ' ')
            #doc_text=doc_text.replace('[**', ' ')
            #doc_text=doc_text.replace('**]', ' ')
            doc_text=doc_text.lower()
                        
            doc_class, context_doc, annotations, relations = myPipe.process(doc_text)
            
            results[i] = doc_class
            context_doc_res.append(context_doc)
            doc_annotations[i] = annotations
            
     

## Save annotation dataframes to annotation files

In [7]:

for k in doc_annotations:
        
    k0=k.split('.')[0]
    k1=k0+'.nlp'

    nlp_ann=''
    for index, row in doc_annotations[k].iterrows():        
        
        nlp_ann = nlp_ann + str(row['markup_id']) +'\t'  
        nlp_ann = nlp_ann + str(row['type']) +' '
        nlp_ann = nlp_ann + str(row['start']) +' '
        nlp_ann = nlp_ann + str(row['end']) +'\t'
        nlp_ann = nlp_ann + str(row['txt'])  +'\n'

    nlpann='nlpann/'+k1
    with open(nlpann, 'w') as myfile:
        myfile.write(nlp_ann)                
    
### select one note/doc for mention level evaluation later ***    
    if k == '23803_184.txt':
        ann1=doc_annotations[k]
#ann1

## Print classification results

In [8]:
for j in results:
    print(j, " ----- ", results[j])


90688_292.txt  -----  no_pe
65675_64.txt  -----  no_pe
48640_63.txt  -----  no_pe
86087_123.txt  -----  pe_doc
83838_106.txt  -----  no_pe
72554_306.txt  -----  no_pe
15899_182.txt  -----  pe_doc
13867_266.txt  -----  pe_doc
61180_73.txt  -----  no_pe
32113_141.txt  -----  no_pe
59381_293.txt  -----  no_pe
58515_159.txt  -----  pe_doc
6878_279.txt  -----  no_pe
820_14.txt  -----  pe_doc
32113_109.txt  -----  pe_doc
49079_68.txt  -----  pe_doc
10568_20.txt  -----  no_pe
25764_268.txt  -----  no_pe
1498_225.txt  -----  no_pe
82326_55.txt  -----  pe_doc


## Document Level Validation

In [9]:
path = path
posLab = "pe_doc"   #positive_DOC
negLab = "no_pe"    #negative_DOC

In [10]:
from pynlp_valid import Validnote

In [11]:
validnote = Validnote()
std_doc = validnote.readstd_ann(path, posLab, negLab)   #readstd(posPath, negPath, posLab, negLab)

In [12]:
for t in std_doc:
    print(t, " ----- ", std_doc[t])

83838_106.txt  -----  no_pe
13867_266.txt  -----  pe_doc
6878_279.txt  -----  no_pe
10568_20.txt  -----  no_pe
820_14.txt  -----  pe_doc
72554_306.txt  -----  no_pe
1498_225.txt  -----  no_pe
86087_123.txt  -----  pe_doc
32113_109.txt  -----  pe_doc
65675_64.txt  -----  no_pe
15899_182.txt  -----  pe_doc
59381_293.txt  -----  pe_doc
58515_159.txt  -----  pe_doc
90688_292.txt  -----  pe_doc
25764_268.txt  -----  no_pe
82326_55.txt  -----  pe_doc
48640_63.txt  -----  no_pe
49079_68.txt  -----  pe_doc
61180_73.txt  -----  no_pe
32113_141.txt  -----  no_pe


In [13]:
precision, recall, f1 = validnote.validation(results, std_doc, posLab, negLab)
print("*"*20)
print("Precision: ", precision)
print("Recall: ", recall)
print("F1: ", f1)

fn --- 90688_292.txt
fn --- 59381_293.txt
	Reference 	 Total
System 	 8 	 0 	 8
System 	 2 	 10 	 12
Total 	 10 	 10
********************
Precision:  1.0
Recall:  0.8
F1:  0.888888888888889


## Mention Level Evaluation

In [14]:
with open(os.path.join(path,'90688_292.txt'),encoding=doc_txt["encoding"]) as f:
    doc_text1 = f.read() 
    doc_text1=doc_text1.lower()
    doc_class, context_doc, annotations, relations = myPipe.process(doc_text1)

In [15]:
# before test, change text filename and run above cell again 
doc_annotations["90688_292.txt"]

,markup_id,vis_category,start,end,txt,type
0,T0,Target,127.0,129.0,pe,pe
1,T1,Target,423.0,425.0,pe,pe
2,T2,Target,475.0,477.0,pe,pe
3,T3,Target,708.0,726.0,pulmonary embolism,pe
4,T4,Modifier,464.0,474.0,no central,definite_negated_existence
5,T5,Target,1225.0,1242.0,pulmonary embolus,pe
6,T6,Target,1531.0,1533.0,pe,pe
7,T7,Modifier,1253.0,1267.0,no evidence of,definite_negated_existence
8,T8,Target,2032.0,2050.0,pulmonary embolism,pe
9,T9,Target,2220.0,2237.0,pulmonary embolus,pe


In [16]:
#test
relations

,relation_id,type,arg1_cate,arg1_id,arg2_cate,arg2_id
0,R0,definite_negated_existence,Modifier,T4,Target,T2
1,R1,definite_negated_existence,Modifier,T4,Target,T3
2,R2,definite_negated_existence,Modifier,T7,Target,T6
3,R3,definite_negated_existence,Modifier,T10,Target,T9


In [17]:
#test
annotations_idx = annotations.set_index('markup_id')
annotations_idx

,vis_category,start,end,txt,type
markup_id,,,,,
T0,Target,127.0,129.0,pe,pe
T1,Target,423.0,425.0,pe,pe
T2,Target,475.0,477.0,pe,pe
T3,Target,708.0,726.0,pulmonary embolism,pe
T4,Modifier,464.0,474.0,no central,definite_negated_existence
T5,Target,1225.0,1242.0,pulmonary embolus,pe
T6,Target,1531.0,1533.0,pe,pe
T7,Modifier,1253.0,1267.0,no evidence of,definite_negated_existence
T8,Target,2032.0,2050.0,pulmonary embolism,pe


In [18]:
#test
target_markups = annotations[annotations['vis_category'] == 'Target']
target_markups

,markup_id,vis_category,start,end,txt,type
0,T0,Target,127.0,129.0,pe,pe
1,T1,Target,423.0,425.0,pe,pe
2,T2,Target,475.0,477.0,pe,pe
3,T3,Target,708.0,726.0,pulmonary embolism,pe
5,T5,Target,1225.0,1242.0,pulmonary embolus,pe
6,T6,Target,1531.0,1533.0,pe,pe
8,T8,Target,2032.0,2050.0,pulmonary embolism,pe
9,T9,Target,2220.0,2237.0,pulmonary embolus,pe
11,T11,Target,2884.0,2902.0,pulmonary embolism,pe
12,T12,Target,3597.0,3599.0,pe,pe


In [19]:
targets = set(target_markups['markup_id'].tolist())
targets

{'T0', 'T1', 'T11', 'T12', 'T13', 'T2', 'T3', 'T5', 'T6', 'T8', 'T9'}

## read ann and convert to df
## check the <span style="color:red">fp</span> and <span style="color:red">fn</span>

In [20]:
import os
unid = 'u0496358'
path = "/home/"+str(unid)+"/BRAT/"+str(unid)+"/Project_pe_test" 

ann_file='90688_292.ann' 

In [21]:
# read ann and convert to df

import numpy as np
import pandas as pd

annoList = []

with open(os.path.join(path,ann_file)) as f:
    ann_file = f.read()
ann_file=ann_file.split('\n')

for line in ann_file:

    if(line.startswith('T')):
        line=line.replace('\n', '')
        line=line.split('\t')

        line0=line[0]
        line2=line[2]
        line1=line[1].split(' ')
        
        if (';' in line1[2]):
            line1.remove(line1[2])   # remove middle span of annotated phrase seprated in 2 line, keep the annotation.
                        
        annList = []
        annList.append(line[0])
        annList.extend(line1)
        annList.append(line[2])
        annoList.append(annList)
#print(annoList)  
    
ann_df = pd.DataFrame(annoList, columns=['markup_id','type','start','end','txt'])    
ann_df


,markup_id,type,start,end,txt
0,T1,High_Ddimer,398,414,elevated D-dimer
1,T2,High_Ddimer,2023,2039,elevated D-dimer
2,T3,PE,154,156,PE
3,T4,PE,459,461,PE
4,T5,PE,688,690,PE
5,T6,PE,3154,3156,PE
6,T7,PE,653,655,PE
7,T8,PE,6332,6334,PE
8,T9,PE,929,947,pulmonary embolism
9,T11,PE,4648,4666,pulmonary embolism


## convert df to annotation object, compare two annotations

In [22]:
def df2ann(df=[], pdoc_type='', ndoc_type=''):
    from pipeUtils import Annotation
    from pipeUtils import Document
    
    Annotations1=[]
    for index, row in df.iterrows() :

        if (pdoc_type == row['type'] or ndoc_type == row['type']):
            continue
        ann_obj=Annotation(start_index=row['start'], end_index=row['end'], type=row['type'], spanned_text=row['txt'], ann_id=row['markup_id'])
        Annotations1.append(ann_obj)

    return Annotations1

###############################################################################################

def compare2ann_types_by_span(ref_ann=[], sys_ann=[], ref_type ='Annotation', sys_type='Annotation', exact=True):
    tp, fp, fn = 0,0,0
    tp_list = []
    fp_list = []
    fn_list = []
    ref_anns = []
    sys_anns = []

    # Split annotations of different types into two lists
    for a in ref_ann:
        if(a.type == ref_type):
            ref_anns.append(a)
    for a in sys_ann:
        if(a.type == sys_type):
            sys_anns.append(a)

    # Count tp and fp
    for sys_ann in sys_anns:
        tp_flag = False
        matching_ref = None
        for ref_ann in ref_anns:
            if exact:
                if(sys_ann.exactMatch(ref_ann)):
                    tp_flag=True
                    matching_ref = ref_ann
            else:
                if sys_ann.overlaps(ref_ann):
                    tp_flag = True
                    matching_ref = ref_ann
        if tp_flag:
            tp = tp + 1
            tp_list.append([sys_ann, matching_ref])
        else:
            fp = fp + 1
            fp_list.append(sys_ann)

    # Count fn
    for ref_ann in ref_anns:
        tp_flag = False
        for sys_ann in sys_anns:
            if exact:
                if(ref_ann.exactMatch(sys_ann)):
                    tp_flag=True
            else:
                if ref_ann.overlaps(sys_ann):
                    tp_flag = True
        if not tp_flag:
            fn = fn + 1
            fn_list.append(ref_ann)

    return tp, fp, fn, tp_list, fp_list, fn_list

In [23]:
Annotations2=df2ann(ann_df, pdoc_type='positive_DOC', ndoc_type='negative_DOC')
for a in Annotations2:
    print (a.ann_id, a.type, a.start_index, a.end_index, a.spanned_text)


T1 High_Ddimer 398 414 elevated D-dimer
T2 High_Ddimer 2023 2039 elevated D-dimer
T3 PE 154 156 PE
T4 PE 459 461 PE
T5 PE 688 690 PE
T6 PE 3154 3156 PE
T7 PE 653 655 PE
T8 PE 6332 6334 PE
T9 PE 929 947 pulmonary embolism
T11 PE 4648 4666 pulmonary embolism
T12 PE 5671 5689 pulmonary embolism
T10 PE 3777 3795 pulmonary embolism
T13 PE 2051 2068 pulmonary embolus
T14 PE 3968 3985 pulmonary embolus


In [24]:
Annotations3=df2ann(annotations, pdoc_type='positive_DOC', ndoc_type='negative_DOC')
for a in Annotations3:
    print (a.ann_id, a.type, a.start_index, a.end_index, a.spanned_text)

T0 pe 127.0 129.0 pe
T1 pe 423.0 425.0 pe
T2 pe 475.0 477.0 pe
T3 pe 708.0 726.0 pulmonary embolism
T4 definite_negated_existence 464.0 474.0 no central
T5 pe 1225.0 1242.0 pulmonary embolus
T6 pe 1531.0 1533.0 pe
T7 definite_negated_existence 1253.0 1267.0 no evidence of
T8 pe 2032.0 2050.0 pulmonary embolism
T9 pe 2220.0 2237.0 pulmonary embolus
T10 definite_negated_existence 2209.0 2219.0 no central
T11 pe 2884.0 2902.0 pulmonary embolism
T12 pe 3597.0 3599.0 pe
T13 pe 3034.0 3052.0 pulmonary embolism


In [25]:
tp, fp, fn, tp_list, fp_list, fn_list = compare2ann_types_by_span(ref_ann=Annotations2, sys_ann=Annotations3, ref_type ='PE', sys_type='pe', exact=True)
print("tp, fp, fn")
print(tp, fp, fn)
print("-----fn-----")
for i in fn_list:
    print(i.ann_id, i.start_index, i.end_index)

tp, fp, fn
0 11 12
-----fn-----
T3 154 156
T4 459 461
T5 688 690
T6 3154 3156
T7 653 655
T8 6332 6334
T9 929 947
T11 4648 4666
T12 5671 5689
T10 3777 3795
T13 2051 2068
T14 3968 3985
